<a href="https://colab.research.google.com/github/rishidur/Helium-Leak-Detection-ML-Model/blob/main/OGI_Image_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Steps to Main Process
# 1 - Temperature Spike /Trough sent to OGI Camera System
# 2 - Camera snaps, sends to ML Model
# 3 - ML Model decides if the problem is a helium gas leak and needs attention
# 4 - ML Model alerts Astronauts

In [ ]:
!pip install tensorflow

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator


train_datagen = ImageDataGenerator(rescale=1./255)
val_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    directory=train_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary'
)

val_generator = val_datagen.flow_from_directory(
    directory=val_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary'
)

print("Training data class labels:", train_generator.class_indices)
print("Validation data class labels:", val_generator.class_indices)


Found 8 images belonging to 2 classes.
Found 6 images belonging to 2 classes.
Training data class labels: {'.ipynb_checkpoints': 0, 'class2': 1}
Validation data class labels: {'.ipynb_checkpoints': 0, 'class1': 1}


In [ ]:
print("Training data class labels:", train_generator.class_indices)
print("Validation data class labels:", val_generator.class_indices)

Training data class labels: {'.ipynb_checkpoints': 0, 'class2': 1}
Validation data class labels: {'.ipynb_checkpoints': 0, 'class1': 1}


In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.optimizers import Adam


train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

val_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory (
    directory='/content/training_data',
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary'
)

val_generator = val_datagen.flow_from_directory(
    directory='/content/validation_data',
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary'
)


model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(512, activation='relu'),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=['accuracy'])


history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=15
)

print("Training Accuracy:", history.history['accuracy'][-1])
print("Validation Accuracy:", history.history['val_accuracy'][-1])

Found 8 images belonging to 2 classes.
Found 6 images belonging to 2 classes.
Epoch 1/15
1/1 ━━━━━━━━━━━━━━━━━━━━ 7s 7s/step - accuracy: 1.0000 - loss: 0.6132 - val_accuracy: 1.0000 - val_loss: 1.3125e-25
Epoch 2/15
1/1 ━━━━━━━━━━━━━━━━━━━━ 8s 8s/step - accuracy: 1.0000 - loss: 1.4458e-08 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 3/15
1/1 ━━━━━━━━━━━━━━━━━━━━ 7s 7s/step - accuracy: 1.0000 - loss: 1.8963e-15 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 4/15
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step - accuracy: 1.0000 - loss: 6.0091e-36 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 5/15
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 6/15
1/1 ━━━━━━━━━━━━━━━━━━━━ 5s 5s/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 7/15
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 8/1

In [ ]:
from tensorflow.keras.models import load_model


loaded_model = load_model('/content/gas_leak_model.h5')


In [ ]:
class_labels = {0: 'no_gas_leak', 1: 'gas_leak'}

In [ ]:
from tensorflow.keras.preprocessing import image
import numpy as np

def preprocess_image(img_path):
    img = image.load_img(img_path, target_size=(224, 224))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array /= 255.0
    return img_array



1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 182ms/step
Predicted class: class2
Prediction probabilities: [[0. 1.]]


In [ ]:
def predict_gas_leak(model, img_path):
    img_array = preprocess_image(img_path)
    predictions = model.predict(img_array)
    predicted_class = np.argmax(predictions, axis=1)[0]


    class_labels = {0: 'no_gas_leak', 1: 'gas_leak'}
    result = class_labels.get(predicted_class, 'Unknown')

    return result, predictions[0][predicted_class]


img_path = '/content/path_to_your_image.jpg'
result, probability = predict_gas_leak(loaded_model, img_path)

print(f"Prediction: {result}")
print(f"Probability: {probability:.2f}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 674ms/step - accuracy: 1.0000 - loss: 0.0000e+00
Validation Loss: 0.0
Validation Accuracy: 1.0


In [ ]:
from tensorflow.keras.preprocessing import image
import numpy as np
from tensorflow.keras.models import load_model


model = load_model('/content/gas_leak_model.h5')


class_labels = {0: 'no_gas_leak', 1: 'gas_leak'}

def preprocess_image(img_path):
    img = image.load_img(img_path, target_size=(224, 224))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array /= 255.0
    return img_array

def predict_gas_leak(model, img_path):
    img_array = preprocess_image(img_path)
    predictions = model.predict(img_array)
    predicted_class = np.argmax(predictions, axis=1)[0]
    class_labels = {0: 'nNo Gas Leak Detected', 1: 'Leaking Gas Detected Alert!'}
    result = class_labels.get(predicted_class, 'Unknown')
    return result, predictions[0][predicted_class]


model = load_model('/content/gas_leak_model.h5')


result, probability = predict_gas_leak(model, '/content/GasLeakPhoto.png')  # Overfittting due to smaller size datasets
print(f"Prediction: {result}, Probability: {probability:.2f}")



1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 147ms/step
Prediction: Leaking Gas Detected Alert!, Probability: 1.00
